In [1]:
import sys
from os.path import isdir, isfile
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from decimal import Decimal

# adding ShadowGrouping package to the system path
# SG_package_path = r"C:\\Users\\bpmur\\OneDrive\\Microsoft Teams Chat Files\\Documents\\Work\\Research\\Numerics\\ShadowGrouping Code\\"
SG_package_path = r"C:\\Users\\Bruno Murta\\OneDrive\\Microsoft Teams Chat Files\\Documents\\Work\\Research\\Numerics\\ShadowGrouping Code\\"
sys.path.insert(0, SG_package_path)

# defining path of folder where molecular Hamiltonians are stored
folder_Hamiltonians = SG_package_path + "Hamiltonians\\"

# defining path where OGM probabilities are stored
folder_OGM_settings = SG_package_path + "OGM_probabilities\\OGM_{}_{}{}.txt" # format string to fill in {molecule}x{qubit_number}x{mapping}

Importing original version of ShadowGrouping code (v1)

In [2]:
# load the measurement schemes from the Benchmark
from shadowgrouping.measurement_schemes import Shadow_Grouping, Derandomization, AdaptiveShadows, L1_sampler, hit_by, N_delta
from shadowgrouping.measurement_schemes_Shadowupdate import Shadow_Grouping_Update, Shadow_Grouping_Update2, Shadow_Grouping_Update3, Shadow_Grouping_Update4, Shadow_Grouping_Update6, Shadow_Grouping_Update7
from shadowgrouping.measurement_schemes import SettingSampler as Overlapped_Grouping
from shadowgrouping.AEQuO import AEQuO
from shadowgrouping.weight_functions import Inconfidence_bound, Bernstein_bound

# wrapper class to combine the measurement scheme with the respective outcomes
from shadowgrouping.energy_estimator import Energy_estimator, StateSampler, Sign_estimator

# helper functions to load Hamiltonian decompositions
from shadowgrouping.measurement_schemes import setting_to_str
from shadowgrouping.hamiltonian import get_pauli_list, get_groundstate, char_to_int, int_to_char, mappings, load_pauli_list

Choosing molecule, basis set and fermion-to-qubit mapping

In [3]:
molecule_name = "LiH" # choose one out of the molecules ['H2', 'H2_6-31g', 'LiH', 'BeH2', 'H2O', 'NH3']
mapping_name = "JW" # choose one out of ["JW","BK","Parity"]
basis_set = "sto3g" # choose one out of ["sto3g","6-31g"] - the latter only for H2 molecule

Obtaining Pauli decomposition of Hamiltonian and its exact ground state from saved data

In [4]:
observables, w, offset, E_GS, state = load_pauli_list(folder_Hamiltonians,molecule_name,basis_set,mapping_name,verbose=True,sparse=True)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\\\Users\\\\Bruno Murta\\\\OneDrive\\\\Microsoft Teams Chat Files\\\\Documents\\\\Work\\\\Research\\\\Numerics\\\\ShadowGrouping Code\\\\Hamiltonians\\'

Initializing 'method' that will generate measurement scheme to estimate the energy

In [5]:
def method_init(method_name, molecule_name, mapping_name, observables, w, offset, eps):
    if method_name == 'ShadowGrouping':
        alpha = np.max(np.abs(w))/np.min(np.abs(w)) + np.min(np.abs(w))
        method = Shadow_Grouping(observables,w,eps,Bernstein_bound(alpha=alpha)())
    elif method_name == 'Derandomization':
        method = Derandomization(observables,w,np.sqrt(0.9),use_one_norm=True)
    elif method_name == 'RandomPaulis':
        method = Derandomization(observables,w,eps,delta=1) # delta controls the randomness
    elif method_name == 'AdaptivePaulis':
        method = AdaptiveShadows(observables,w)
    elif method_name == 'AEQuO':
        # values from figure 5 (L == adaptiveness_L +1) of https://arxiv.org/abs/2110.15339
        method = AEQuO(observables,w,offset,adaptiveness_L=2,interval_skewness_l=4,budget=1000)
    elif method_name == 'OverlappedGrouping':
        # catch exception of missing data for OGM
        file = folder_OGM_settings.format(molecule_name,observables.shape[1],mapping_name.lower())
        if isfile(file):
            method = Overlapped_Grouping(observables,w,file)
    else:
        print('method_name provided is invalid.')
        return False

    return method

In [6]:
method_name = 'ShadowGrouping'
method = method_init(method_name, molecule_name, mapping_name, observables, w, offset, eps=0.1)

Initializing 'estimator', which executes the three key parts:

1. generation of measurement settings via 'propose_next_settings', which calls find_setting from chosen 'measurement_scheme' method
2. sampling of reference state via StateSampler
3. computation of energy via its own 'measure' and 'get_energy' functions

In [7]:
estimator = Energy_estimator(method,StateSampler(state),offset=offset)

Printing the number of observables

In [8]:
print(len(observables))

630


Generating measurement scheme with N_rounds and printing number of occurrences of each setting

In [9]:
estimator.reset()
N_rounds = 142
estimator.propose_next_settings(N_rounds)
# version 1
print(estimator.settings_dict)

{'XXXXXXXXYYYY': 1, 'XXXXYYYYXXXX': 1, 'XXXZZXXZXZZX': 1, 'XXXZZXXZZZZX': 1, 'XXXZZXYYZZZY': 1, 'XXYZZYYZYZZY': 1, 'XXZXXZZZXXXZ': 1, 'XXZZZXXXXZZX': 1, 'XXZZZXZXZZZX': 1, 'XXZZZXZYYZZY': 1, 'XXZZZZYZZZZY': 1, 'XYYZZXYYXZZX': 1, 'XZXZXXXZXZXX': 1, 'XZXZXZXXZZXZ': 1, 'XZXZXZYYZZYZ': 1, 'XZXZZXYZYZZY': 1, 'XZXZZXZXXZZX': 1, 'XZXZZZXYYZZX': 1, 'XZXZZZXZZZZX': 1, 'XZXZZZYZZZZY': 1, 'XZXZZZZXZZZX': 1, 'XZXZZZZYZZZY': 1, 'XZZXZXZZXXZX': 1, 'XZZXZZXZZXZZ': 1, 'XZZXZZYZZYZZ': 1, 'XZZXZZZXZXZZ': 1, 'XZZXZZZYZYZZ': 1, 'XZZXZZZZYYZY': 1, 'XZZZXZXZZZXZ': 1, 'XZZZXZYZZZYZ': 1, 'XZZZXZZXZZXZ': 1, 'XZZZXZZYZZYZ': 1, 'XZZZXZZZXZXX': 1, 'XZZZXZZZYZYY': 1, 'XZZZZXXXXZZX': 1, 'XZZZZXXZXZZZ': 1, 'XZZZZXXZZZZX': 1, 'XZZZZXYYZZZZ': 1, 'XZZZZXYZYZZZ': 1, 'XZZZZXYZZZZY': 1, 'XZZZZXZXZZZX': 1, 'XZZZZXZYYZZY': 1, 'XZZZZXZYZZZY': 1, 'YXXZZYXXYZZY': 1, 'YYXZXXYZYZYY': 1, 'YYXZZXXZZZZX': 1, 'YYYYZYXZXXZX': 1, 'YYYYZYYYYYZY': 1, 'YYYZYYXXXZXX': 1, 'YYYZZYXXZZZX': 1, 'YYYZZYYZZZZY': 1, 'YYYZZYZXXZZZ': 1, 'YYZYYZXZXX

Printing the number of measurement rounds, total number of samples, number of observables, and number of samples per observable

In [10]:
print('Total number of measurement rounds: ', estimator.num_settings)
print('Total number of distinct measurement settings: ', len(estimator.settings_dict))
N_samples = np.sum(np.array(estimator.measurement_scheme.N_hits))
print('Total number of samples across all observables: ', N_samples)
print('Total number of observables: ', len(observables))
print('Number of samples for each observable')
print(estimator.measurement_scheme.N_hits)

Total number of measurement rounds:  142
Total number of distinct measurement settings:  142
Total number of samples across all observables:  6066
Total number of observables:  630
Number of samples for each observable
[ 55  11  12   9   9  11  11   9   8  10  10  75  85 106   9  10 101  54
  12  12  60  11   9   9   9  76   9  10   9   8  86 106   9   9 101  61
   3   3   1   1   1   1   1   1   1   1   3   1   1   1   1   3  11  11
   1   1   1   1   8   7   9   8   7   7   1   1   1   1   1   1   9   8
   9  10   1   1   1   1   9   9   5   6   1   2   1   1   2   2   3   2
   1   1   1   1  12  12   3   3   2   3   2   2  13   1   1   1   1   1
   1   1   1   9  10   1   1   1   1   1   1   1   1  40   1   1   1   1
  37  35  20   1   1   1   1   8   7   4   5   5   5   1   1   1   1  34
   4   4   3   2  38   4   4   1   1   1   1  32   9   8  38   1   2   1
   1   1   1   2   3   1   1   9  10  16  15   1   1   1   1   1   1   1
   1   9   9   4   5   1   1   1   1   4   4   1   

Determining number of observables without any sample

In [11]:
print(len(np.where(estimator.measurement_scheme.N_hits == 0)[0]))

0
